# Contextualize OMArk results with the OMABrowser

This code require to install the [omadb](https://github.com/DessimozLab/pyomadb/) package version 2.2.0 or higher, If did not do it yet, you can do it using the cell below.

In [ ]:
!pip install omadb>=2.2.0

In [ ]:
import pandas as pd
import omark_contextualize as oc

## Extract OMArk data

In [ ]:
#Path to the the OMArk folder
PATH = ''
#Path to the OMAmer file
OMAMER = ''

# Exploring data with a Pandas dataframe

The following cells extract data from the OMArk output file to make their exploration easiers.

The **full_df** dataframe is a combination of consistency and completeness data using the OMAmer mapping. It is possible to filter them by consistent category or completess category.   
Other dataframes are output by this cell that corresponds to the composite cell:   
The **completeness_df** dataframe consists only of completeness results for each HOG.   
The **consistenct_df** dataframe conssists only of consistency assesment for each gene in the proteome.   
The **omamer_df** dataframe is a reproduction of the OMAmer file.   

In [ ]:
full_df, completeness_df, consistency_df, omamer_df = oc.get_data_total(PATH, OMAMER )
level = oc.get_level(PATH)

In [ ]:
full_df

# Fragments analysis

This section of the Notebook is meant to delve into the fragmentary genes in your genome of interests and to help correct them using the OMA Browser data. There are three options depending of your goal, available in the three subsections below. 

**Choose one of the options, run the code then go to export**

Options are:
- All fragments: look for all genes annotated as fragments by OMArk
- Linked fragments: only groups of more than one fragmennts that correspond to the same HOG (Stringent set)
- Fragments and linked genes: look for all fragments as well as other "genes" mapping to the same HOG and smaller than expected (Not small enough to be noted as fragments but may be) - (Extensive set)

In [ ]:
#WARNING - run this
kept_info = ['gene', 'Consistency_Category', 'structure', 'HOG', 'qseqlen', 'subfamily_medianseqlen', 'Completeness_Category']
fragment_df =  full_df[full_df['structure']=='Fragment'][kept_info]


### All fragments

In [ ]:
possible_fragments = fragment_df

### Linked fragments

In [ ]:
possible_fragments = fragment_df[fragment_df['HOG'].duplicated(keep=False)]


### Fragments and linked genes

In [ ]:
len_threshod  = 0.8

hog_with_fragment = list(fragment_df['HOG'].unique())
possible_fragments = full_df[full_df['HOG'].isin(hog_with_fragment)][kept_info].sort_values(by='HOG')
possible_fragments = possible_fragments[possible_fragments['qseqlen']<0.8*possible_fragments['subfamily_medianseqlen']]


## Get sequence  of fragmented HOGs

We uses the OMA API to obtain sequence for the HOGs that corresponds to the fragment to obtain example sequences for those HOGs.   
The next cell extract the unique identifier of each HOG reported as being part of the Fragment set.  
Then each example sequence are downloaded.   
Finally they are written to a FASTA file.

In [ ]:
#Extract uniq HOGs
uniq_HOGs = list(possible_fragments['HOG'].unique())
hog_to_medseqlen  = {k: v for k, v in zip(possible_fragments['HOG'], possible_fragments['subfamily_medianseqlen'])}
hog_genes = {}
for hog, seq in zip(possible_fragments['HOG'],possible_fragments['gene']):
    glist = hog_genes.get(hog, [])
    glist.append(seq)
    hog_genes[hog] =glist
hog_genes
print(f'{len(hog_genes)} different HOGs')

In [ ]:
#Downloading sequences
sequence_of_hog = oc.get_sequences_hog(uniq_HOGs, medseqlen=hog_to_medseqlen)

In [ ]:
# Write a FASTA file with the corresponding sequences. Here is the path to output
SEQ_FASTA =  ''
oc.write_FASTA_fragmented_HOGs(sequence_of_hog, hog_genes, SEQ_FASTA)

### Procedure for fragment correction

The FASTA file provided by the above code contains one (or more, depending of parameters) sequence(s) from each Hierarchical Orthologous Groups for which we detected sequences in the genome were fragmented. The identifier of each sequence is the HOG ID with an added number (In case there are many sequences)

You can use it as an input of MiniProt (https://github.com/lh3/miniprot) to get a mapping of those HOGs to the genomic sequence. We recommend using the ```--gff``` option of MiniProt to obtain the results as the GFF file.  
If MiniProt is installed in yout environment, you can use the below command line to run it, after setting the appropriate pathes.

In [ ]:
#Path for MiniProt
GENOMIC_FASTA  = ''
MINIPROT_OUTPUT = ''

In [ ]:
!miniprot miniprot -I --gff --outs=0.95 {GENOMIC_FASTA} {SEQ_FASTA} > {MINIPROT_OUTPUT}

You can then compare your gene models to the one predicted by MiniProt by homology, and merge them together if need be. 
Both your GFF file and the MiniProt file can be visualized alongside one another using a genome browser, like JBrowse2

# Missing genes analysis

This section of the Notebook is meant to delve into the missing genes in your proteome of interests and to help correct them using the OMA Browser data.   
Here, we offer two complementary ways to help confirm missing genes or find them if they are present in the assembly.
The first one is based on **reference sequences** for the missing HOGs, that can be exploited with methods such as Miniprot to find close sequences in the assembly.   
The second one export information about the **ancestral synteny** of the missing HOGs and is meant to confirm or find these genes are missing by looking at their context.  
  
The folllowing cells get information about the missing HOGs in a dataframe, then extract the unique HOG identifiers for these genes.

In [ ]:
missing_df =  full_df[full_df['Completeness_Category']=='Lost']
missing_df

In [ ]:
uniq_HOGs = list(missing_df['HOG'].unique())

print(f'{len(uniq_HOGs)} different HOGs')

## Sequence validation

This subsection extract reference sequences for the missing HOGs from this annotation and extract reference sequences for this from the OMA Browser, and write it as a FASTA file.
This FASTA file can the be used to do mapping to the genome

In [ ]:
sequence_of_hog = oc.get_sequences_hog(uniq_HOGs, level=level)

In [ ]:
#Path to the output FASTA file
SEQ_FASTA=  ''

In [ ]:
oc.write_FASTA_missing_HOGs(sequence_of_hog, SEQ_FASTA)

The FASTA file provided by the above code contains Hierarchical Orthologous Groups for which we can't find a correspoding proteins in the protromr. The identifier of each sequence is the HOG ID with an added number (In case there are many sequences)

You can use it as an input of MiniProt (https://github.com/lh3/miniprot) to get a mapping of those HOGs to the genomic sequence. We recommend using the ```--gff``` option of MiniProt to obtain the results as the GFF file.  
If MiniProt is installed in yout environment, you can use the below command line to run it, after setting the appropriate pathes.

In [ ]:
#Path for MiniProt
GENOMIC_FASTA  = ''
MINIPROT_OUTPUT = ''

In [ ]:
!miniprot -I --gff --outs=0.95 {GENOMIC_FASTA} {SEQ_FASTA} > {MINIPROT_OUTPUT}

You can then check whether you have gene models predicted at these location, or if the mapping could improve your gene models.  
This method can also be combined with the synteny strategy below

## Synteny analysis

This subsection extract ancestral synteny for the missing HOGs from this annotation, find the corresponding genes in the genome using OMAmer and export this context.
This file can the be used to know if known genes from the same context are present in the genome or also missing, and to help you find these genes if genes from the same context are indeed presents.
The next cells obtain context for the missing HOGs from the OMA API (should take a few minutes), then use the OMAmer mapping to convert it to genes identifier from the same genome and export all as a tsv files.

**Warning** This subsection will only work with OMABrowser version from June 2023 onward.

In [ ]:
#Extract synteny data
synteny_groups = oc.get_synteny_hog(uniq_HOGs, level)

In [ ]:
#Get OMAmer mapping
omamer_map  = {x : y for x,y in zip(list(omamer_df['HOG']),list(omamer_df['gene']) )}

In [ ]:
#Convert context to the target genome context (Gene identifier)
expected_neighbourhood = oc.translate_to_genomic_context(synteny_groups, omamer_map)

In [ ]:
expected_neighbourhood

In [ ]:
#Path to the synteny TSV file
SYNTENY_OUTFILE = ''

In [ ]:
#Write the results to a TSV file
oc.write_synteny_file(expected_neighbourhood, SYNTENY_OUTFILE)

You can use the above generated file to look for the gene close to the "Target" genes (the missing HOG) - represented by their ID in the OMAmer file. YOu can use a genome browser to find this context and identify if the genes may be present, by combining it with the above sequence subsection.

The next cell is histogram of number of genes found in the context for each HOG.  
A high number of gene with 4 genes in the context may mean that these missing genes may still be present in the genome, while a lot of 0 genes in the context likely confirm that entire contigs are missing from the assembly.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.bar(*np.unique([x[1][0] for x in expected_neighbourhood], return_counts=True))


# Assembly completeness assesment

OMARk is designed to assess the quality of annotation, using multiple independant metrics.
However, users may be interested to assess the completeness of an assembly before going through a lengthy annotation process. Here is a tutorial on how to do this.   
This tutorial is divied into three subsection. While the first section is mandatory, the two others are mutually exclusive and are to be done under different conditions:
* Conserved genes extraction with OMArk. **Mandatory**
* Assembly completenes with Miniprot. **Only if the gene set is close enough to the target species (Family level or below**
* Assembly completeness with a genefinder **If the conserved gene set is not close to the target species**


First, you need to extract the reference lineage's HOGs from OMArk. You can do this using the ```-c``` option. Below is the cell that can help you run this command if OMArk is installed in your environment.

In [ ]:
#Path to the OMAmer DB
OMAMER_DB = ''
#Target species's taxid
SPECIES_TAXID = ''
#OMArk folder
OMARK_OUTPUT_FOLDER=  ''

In [ ]:
!omark -c -d {OMAMER_DB} -t {SPECIES_TAXID} -o {OMARK_OUTPUT_FOLDER}

This cell extract the results from the ```omark -c``` call.

In [ ]:
HOG_LIST_FILE = OMARK_OUTPUT_FOLDER + '/conserved_HOGs.txt'

hog_list, LEVEL = oc.read_conserved_hogs(HOG_LIST_FILE)

print(f'Ancestral lineage: {LEVEL}')
print(f'{len(hog_list)} HOGs')

## Assembly completeness with Miniprot

This subsection is to help you assessing the completeness of your assembly rather than an annotation. To do this, we suggest using Miniprot (https://github.com/lh3/miniprot) from [H. Li, *Bioinformatics*, 2023](https://academic.oup.com/bioinformatics/article/39/1/btad014/6989621). 

The cell below download sequences from OMArk's conserved HOGs, at the level of interest. We expect this method to work well when the chosen ancestral lineage is not too far from the species of interest (Order, or family level) since Miniprot works well with close sequences. If it is not the case, we suggest using the Gene Finder subsection below.

**Warning**: downloading selected sequences from OMA require many API calls and can take time. Expect this part to take between 15 min to 4 hours.

In [ ]:
#Input for MiniProt
SEQ_FASTA=  ''
sequence_of_hog = oc.get_sequences_hog(hog_list, nseq=1, level=LEVEL)
oc.write_FASTA_missing_HOGs(sequence_of_hog, SEQ_FASTA)

With this FASTA file, run miniprot with the following command: ```miniprot  --trans -I --gff --outs=0.95 [GENOMIC_FASTA] [SEQ_FASTA] > MINIPROT_OUTPUT```. If miniprot is installed in your environment use the cell below, by replacing values for the empty variables.

In [ ]:
#Path to the genomic FASTA file
GENOMIC_FASTA = ''
#Path to the MiniProt output file
MINIPROT_OUTPUT = ''

In [ ]:
!miniprot  --trans -I --gff --outs=0.95 {GENOMIC_FASTA} {SEQ_FASTA} > {MINIPROT_OUTPUT}

The next cell takes as input the GFF output from Miniprot and output a FASTA file to be used as input for omamer and a splice file to be used as input for OMArk.   
The cell after this gives example command to run OMArk. You may run it through the Notebook if you have it installed in the same environment and have enough ressources for it. Otherwise, you can use it as a command line.

In [ ]:
#FASTA file to be created from MiniProt results
PROT_FASTA = ''
#Splice file to be created from MiniProt Results
SPLICE_FILE = ''
oc.omark_input_from_gff(MINIPROT_OUTPUT, GENOMIC_FASTA, PROT_FASTA, SPLICE_FILE)

In [ ]:
#Path to the OMAmer file
OMAMER_FILE = ''
#Path to OMArk output
OMARK_OUTPUT  = ''

In [ ]:
!omamer search --db {OMAMER_DB} --query {PROT_FASTA} --out {OMAMER_FILE} 
!omark -d {OMAMER_DB} -f {OMAMER_FILE} -i {SPLICE_FILE} -o {OMARK_OUTPUT}

OMArk results give an estimate for the completess of the assembly. Only the completeness part of OMArk is relevant here, since the annotation done this way only consider genes annotated from mapping conserved genes.   
**Warning**: the consistency part of OMArk should not be used to evaluate proteomes directly annotated from OMArk output since it would be circular logic. Instead, consistency evaluation should only be used on annotation created from methods independant from OMArk.

## Assembly completeness with a genefinder

In case the ancestral lineage chosen by OMArk for your proteome is quite distant from your species of interest, it is best to use an *ab initio* predicttion method to first estimate the completeness of the assembly.

The cell below can help you by downloading sequences from the selected lineage to use as a training set for methods such as [Augustus](https://github.com/Gaius-Augustus/Augustus). 

In [ ]:
#input for Genefinder
PATH_OUTPUT=  ''
sequence_of_hog = oc.get_sequences_hog(hog_list, nseq=20, level=LEVEL)
oc.write_FASTA_missing_HOGs(sequence_of_hog, PATH_OUTPUT)

Using *ab initio* annotation method can require parallelizing, or parameter tuning, that is best made following instructions from the authors of a method. Once predictions are made, you can use OMAmer and OMArk in their normal on the output protein FASTA file.